In [1]:
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

```{autolink-concat}
```

::::{margin}
:::{card}  Visualization of the Riemann sheets for the two-channel $T$-matrix with one resonance pole
TR-028
^^^
Following **[TR-026](./026.ipynb)**, the Riemann sheets for the amplitude calculated within the $K$ matrix formalism for the two-channel case are visualized. For that the method of transitioning from the first physical sheet to the unphysical sheets is expanded to the two dimensional case according to [Eur. Phys. J. C (2023) 83:850](https://juser.fz-juelich.de/record/1017534/files/s10052-023-11953-6.pdf?version=1) to also visualize the third and the fourth unphysical sheet.   
:::
::::

# Coupled channel Riemann sheets with $K$ matrix


In [2]:
%pip install -q ampform==0.14.10 plotly==5.18.0 sympy==1.12

Note: you may need to restart the kernel to use updated packages.


In [3]:
from __future__ import annotations

import warnings
from typing import Any

import matplotlib.pyplot as plt
import numpy as np
import plotly.graph_objects as go
import sympy as sp
from ampform.io import aslatex
from ampform.sympy import unevaluated
from IPython.display import Math
from plotly.subplots import make_subplots

warnings.filterwarnings("ignore")

In [4]:
@unevaluated(real=False)
class PhaseSpaceFactor(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt((s - ((m1 + m2) ** 2)) * (s - (m1 - m2) ** 2) / s**2)


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr = PhaseSpaceFactor(s, m1, m2)
Math(aslatex({rho_expr: rho_expr.doit(deep=False)}))

<IPython.core.display.Math object>

In [5]:
from ampform.kinematics.phasespace import Kallen


@unevaluated(real=False)
class PhaseSpaceFactorKallen(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho_{{{m1}, {m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return 2 * BreakupMomentum(s, m1, m2) / sp.sqrt(s)


@unevaluated(real=False)
class PhaseSpaceCM(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\rho^\mathrm{{CM}}_{{{m1},{m2}}}\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return -16 * sp.pi * sp.I * ChewMandelstam(s, m1, m2)


@unevaluated(real=False)
class ChewMandelstam(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"\Sigma\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        q = BreakupMomentum(s, m1, m2)
        return (
            1
            / (16 * sp.pi**2)
            * (
                (2 * q / sp.sqrt(s))
                * sp.log((m1**2 + m2**2 - s + 2 * sp.sqrt(s) * q) / (2 * m1 * m2))
                - (m1**2 - m2**2) * (1 / s - 1 / (m1 + m2) ** 2) * sp.log(m1 / m2)
            )
        )


@unevaluated(real=False)
class BreakupMomentum(sp.Expr):
    s: Any
    m1: Any
    m2: Any
    _latex_repr_ = R"q\left({s}\right)"

    def evaluate(self) -> sp.Expr:
        s, m1, m2 = self.args
        return sp.sqrt(Kallen(s, m1**2, m2**2)) / (2 * sp.sqrt(s))


s, m1, m2 = sp.symbols("s m1 m2")
rho_expr_kallen = PhaseSpaceFactorKallen(s, m1, m2)
rho_cm_expr = PhaseSpaceCM(s, m1, m2)
cm_expr = ChewMandelstam(s, m1, m2)
q_expr = BreakupMomentum(s, m1, m2)
kallen = Kallen(*sp.symbols("x:z"))
Math(
    aslatex({
        e: e.doit(deep=False)
        for e in [rho_expr_kallen, rho_cm_expr, cm_expr, q_expr, kallen]
    })
)

<IPython.core.display.Math object>

## T matrix definition with K matrix - 2 channel case

In the following the dynamical part of the Amplitude calculated within the K matrix formalism.

## First Riemann sheet 

In [6]:
class DiagonalMatrix(sp.DiagonalMatrix):
    def _latex(self, printer, *args):
        return printer._print(self.args[0])


n = 2
I = sp.Identity(n)
K = sp.MatrixSymbol("K", n, n)
CM = DiagonalMatrix(sp.MatrixSymbol(R"\rho^\Sigma", n, n))
Math(aslatex({CM: CM.as_explicit()}))

<IPython.core.display.Math object>

In [7]:
T_I = (I - sp.I * K * CM).inv() * K
T_I

(I - I*K*DiagonalMatrix(\rho^\Sigma))**(-1)*K

In [8]:
T_I_explicit = T_I.as_explicit()
T_I_explicit[0, 0].simplify(doit=False)

((I*K[1, 1]*\rho^\Sigma[1, 1] - 1)*K[0, 0] - I*K[0, 1]*K[1, 0]*\rho^\Sigma[1, 1])/(K[0, 0]*K[1, 1]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] + I*K[0, 0]*\rho^\Sigma[0, 0] - K[0, 1]*K[1, 0]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] + I*K[1, 1]*\rho^\Sigma[1, 1] - 1)

As used in the PDG resonance section a different form of the $K$ matrix is used containing the the $g^{0}$ couplings instet of the unitless $\gamma$ constants.

In [13]:
s = sp.Symbol("s")
ma1 = sp.Symbol("m_{a1}")
mb1 = sp.Symbol("m_{b1}")
ma2 = sp.Symbol("m_{a2}")
mb2 = sp.Symbol("m_{b2}")
m0 = sp.Symbol("m0")
w0 = sp.Symbol("Gamma0")
g1 = sp.Symbol("g1")
g2 = sp.Symbol("g2")
symbols = sp.Tuple(s, ma1, mb1, ma2, mb2, m0, w0, gamma1, gamma2)
k_expr_00 = (g1*g1) / (s - m0**2)
k_expr_10 = (g1*g2) / (s - m0**2)
k_expr_11 = (g2*g2) / (s - m0**2)

In [14]:
cm_expressions = {
    K[0, 0]: k_expr_00,
    K[1, 1]: k_expr_11,
    K[0, 1]: k_expr_10,
    K[1, 0]: k_expr_10,
    CM[0, 0]: -PhaseSpaceCM(s, ma1, mb1),
    CM[1, 1]: -PhaseSpaceCM(s, ma2, mb2),
}

In [15]:
T_I_cm_expr = T_I_explicit.xreplace(cm_expressions)
T_I_cm_expr[0, 0].simplify(doit=False)

g1**2/(I*g1**2*PhaseSpaceCM(s, m_{a1}, m_{b1}) + I*g2**2*PhaseSpaceCM(s, m_{a2}, m_{b2}) - m0**2 + s)

## Second, third and fourth Riemann sheet 

For the case of two channels there are four Riemann sheets. The first physical one and three unphysical ones. The physical sheet is calculated using the analytic solution of the Chew-Mandelstam function. 

$$
\begin{eqnarray}
\operatorname{Disc}_{\mathrm{I,II}} T_K^{-1}
&=& 2 i\left[\begin{array}{rr}\rho_1 & 0  \\ 0 & 0 \end{array}\right], \\
\operatorname{Disc}_{\mathrm{I,III}} T_K^{-1}
&=& 2 i\left[\begin{array}{rr}\rho_1 & 0  \\ 0 & \rho_2 \end{array}\right], \\
\operatorname{Disc}_{\mathrm{I,IV}} T_K^{-1}
&=& 2 i\left[\begin{array}{rr}0 & 0  \\ 0& \rho_2  \end{array}\right].
\end{eqnarray}
$$

Depending on the centre-of-mass energy, different Riemann sheets connect smoothly to the physical one. Therefore, two cases are studied: one where the resonance mass is above the threshold of the second and first channel, and another where the resonance mass is between the threshold of the first and second channel.

In [16]:
n = 2
rho = DiagonalMatrix(sp.MatrixSymbol("rho", n, n))
Math(aslatex({rho: rho.as_explicit()}))

<IPython.core.display.Math object>

In [17]:
T_II = (T_I.inv() + 2 * sp.I * rho).inv()
T_III = (T_I.inv() + 2 * sp.I * rho).inv()
T_IV = (-T_I.inv() - 2 * sp.I * rho).inv()

In [18]:
T_II_explicit = T_II.as_explicit()
T_II_explicit[0, 0].simplify(doit=False)

(-I*K[0, 0]*K[1, 1]*\rho^\Sigma[1, 1] + 2*I*K[0, 0]*K[1, 1]*rho[1, 1] + K[0, 0] + I*K[0, 1]*K[1, 0]*\rho^\Sigma[1, 1] - 2*I*K[0, 1]*K[1, 0]*rho[1, 1])/(-K[0, 0]*K[1, 1]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] + 2*K[0, 0]*K[1, 1]*\rho^\Sigma[0, 0]*rho[1, 1] + 2*K[0, 0]*K[1, 1]*\rho^\Sigma[1, 1]*rho[0, 0] - 4*K[0, 0]*K[1, 1]*rho[0, 0]*rho[1, 1] - I*K[0, 0]*\rho^\Sigma[0, 0] + 2*I*K[0, 0]*rho[0, 0] + K[0, 1]*K[1, 0]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] - 2*K[0, 1]*K[1, 0]*\rho^\Sigma[0, 0]*rho[1, 1] - 2*K[0, 1]*K[1, 0]*\rho^\Sigma[1, 1]*rho[0, 0] + 4*K[0, 1]*K[1, 0]*rho[0, 0]*rho[1, 1] - I*K[1, 1]*\rho^\Sigma[1, 1] + 2*I*K[1, 1]*rho[1, 1] + 1)

In [19]:
T_III_explicit = T_III.as_explicit()
T_III_explicit[0, 0].simplify(doit=False)

(-I*K[0, 0]*K[1, 1]*\rho^\Sigma[1, 1] + 2*I*K[0, 0]*K[1, 1]*rho[1, 1] + K[0, 0] + I*K[0, 1]*K[1, 0]*\rho^\Sigma[1, 1] - 2*I*K[0, 1]*K[1, 0]*rho[1, 1])/(-K[0, 0]*K[1, 1]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] + 2*K[0, 0]*K[1, 1]*\rho^\Sigma[0, 0]*rho[1, 1] + 2*K[0, 0]*K[1, 1]*\rho^\Sigma[1, 1]*rho[0, 0] - 4*K[0, 0]*K[1, 1]*rho[0, 0]*rho[1, 1] - I*K[0, 0]*\rho^\Sigma[0, 0] + 2*I*K[0, 0]*rho[0, 0] + K[0, 1]*K[1, 0]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] - 2*K[0, 1]*K[1, 0]*\rho^\Sigma[0, 0]*rho[1, 1] - 2*K[0, 1]*K[1, 0]*\rho^\Sigma[1, 1]*rho[0, 0] + 4*K[0, 1]*K[1, 0]*rho[0, 0]*rho[1, 1] - I*K[1, 1]*\rho^\Sigma[1, 1] + 2*I*K[1, 1]*rho[1, 1] + 1)

In [20]:
T_IV_explicit = T_IV.as_explicit()
T_IV_explicit[0, 0].simplify(doit=False)

(I*K[0, 0]*K[1, 1]*\rho^\Sigma[1, 1] - 2*I*K[0, 0]*K[1, 1]*rho[1, 1] - K[0, 0] - I*K[0, 1]*K[1, 0]*\rho^\Sigma[1, 1] + 2*I*K[0, 1]*K[1, 0]*rho[1, 1])/(-K[0, 0]*K[1, 1]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] + 2*K[0, 0]*K[1, 1]*\rho^\Sigma[0, 0]*rho[1, 1] + 2*K[0, 0]*K[1, 1]*\rho^\Sigma[1, 1]*rho[0, 0] - 4*K[0, 0]*K[1, 1]*rho[0, 0]*rho[1, 1] - I*K[0, 0]*\rho^\Sigma[0, 0] + 2*I*K[0, 0]*rho[0, 0] + K[0, 1]*K[1, 0]*\rho^\Sigma[0, 0]*\rho^\Sigma[1, 1] - 2*K[0, 1]*K[1, 0]*\rho^\Sigma[0, 0]*rho[1, 1] - 2*K[0, 1]*K[1, 0]*\rho^\Sigma[1, 1]*rho[0, 0] + 4*K[0, 1]*K[1, 0]*rho[0, 0]*rho[1, 1] - I*K[1, 1]*\rho^\Sigma[1, 1] + 2*I*K[1, 1]*rho[1, 1] + 1)

In [21]:
rho_expressions_II = {
    **cm_expressions,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    rho[1, 1]: 0,
}
rho_expressions_III = {
    **cm_expressions,
    rho[0, 0]: PhaseSpaceFactor(s, ma1, mb1),
    rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}
rho_expressions_IV = {
    **cm_expressions,
    rho[0, 0]: 0,
    rho[1, 1]: PhaseSpaceFactor(s, ma2, mb2),
}

In [22]:
T_II_rho_expr = T_II_explicit.xreplace(rho_expressions_II)
T_III_rho_expr = T_III_explicit.xreplace(rho_expressions_III)
T_IV_rho_expr = T_IV_explicit.xreplace(rho_expressions_IV)

In [23]:
T_II_rho_expr[0, 0].simplify(doit=False)

g1**2/(I*g1**2*PhaseSpaceCM(s, m_{a1}, m_{b1}) + 2*I*g1**2*PhaseSpaceFactor(s, m_{a1}, m_{b1}) + I*g2**2*PhaseSpaceCM(s, m_{a2}, m_{b2}) - m0**2 + s)

In [24]:
T_III_rho_expr[0, 0].simplify(doit=False)

g1**2/(I*g1**2*PhaseSpaceCM(s, m_{a1}, m_{b1}) + 2*I*g1**2*PhaseSpaceFactor(s, m_{a1}, m_{b1}) + I*g2**2*PhaseSpaceCM(s, m_{a2}, m_{b2}) + 2*I*g2**2*PhaseSpaceFactor(s, m_{a2}, m_{b2}) - m0**2 + s)

## Visualization of the 2 dimensional lineshape 

In [ ]:
T_I_func = sp.lambdify(symbols, T_I_cm_expr[0, 0].doit())
T_II_func = sp.lambdify(symbols, T_II_rho_expr[0, 0].doit())
T_III_func = sp.lambdify(symbols, T_III_rho_expr[0, 0].doit())
T_IV_func = sp.lambdify(symbols, T_IV_rho_expr[0, 0].doit())
values_1 = {
    ma1: 0.14,
    mb1: 0.14,
    ma2: 0.5,
    mb2: 0.5,
    m0: 0.980,
    w0: 0.5,
    g1: -np.sqrt(0.076),
    g2:  np.sqrt(0.186),
}
values_2 = {
    **values_1,
    m0: 1.1,
}

args_1 = eval(str(symbols[1:].xreplace(values_1)))
args_2 = eval(str(symbols[1:].xreplace(values_2)))

epsilon = 1e-5
x = np.linspace(0, 8, num=200)
y = np.linspace(epsilon, 1, num=100)
X, Y = np.meshgrid(x, y)
Zn = X - Y * 1j
Zp = X + Y * 1j

T1n_res1 = T_I_func(Zn**2, *args_1)
T1p_res1 = T_I_func(Zp**2, *args_1)

T2n_res1 = T_II_func(Zn**2, *args_1)
T2p_res1 = T_II_func(Zp**2, *args_1)

T3n_res1 = T_III_func(Zn**2, *args_1)
T3p_res1 = T_III_func(Zp**2, *args_1)

T4n_res1 = T_IV_func(Zn**2, *args_1)
T4p_res1 = T_IV_func(Zp**2, *args_1)

T1n_res2 = T_I_func(Zn**2, *args_2)
T1p_res2 = T_I_func(Zp**2, *args_2)

T2n_res2 = T_II_func(Zn**2, *args_2)
T2p_res2 = T_II_func(Zp**2, *args_2)

T3n_res2 = T_III_func(Zn**2, *args_2)
T3p_res2 = T_III_func(Zp**2, *args_2)

T4n_res2 = T_IV_func(Zn**2, *args_2)
T4p_res2 = T_IV_func(Zp**2, *args_2)

fig, axes = plt.subplots(figsize=(11, 6), ncols=4, sharey=True)
ax1, ax2, ax3, ax4 = axes
for ax in axes:
    ax.set_xlabel(R"$\mathrm{Re}(s)$")

ax1.plot(x, T1n_res1[0].imag, label=R"$T_\mathrm{I}(s-0i)$")
ax1.plot(x, T1p_res1[0].imag, label=R"$T_\mathrm{I}(s+0i)$")
ax1.set_title(f"${sp.latex(rho_cm_expr)}$")
ax1.set_title(R"$T_\mathrm{I}$")

ax2.plot(x, T2n_res1[0].imag, label=R"$T_\mathrm{II}(s-0i)$")
ax2.plot(x, T2p_res1[0].imag, label=R"$T_\mathrm{II}(s+0i)$")
ax2.set_title(R"$T_\mathrm{II}$")

ax3.plot(x, T3n_res1[0].imag, label=R"$T_\mathrm{III}(s-0i)$")
ax3.plot(x, T3p_res1[0].imag, label=R"$T_\mathrm{III}(s+0i)$")
ax3.set_title(R"$T_\mathrm{III}$")

ax4.plot(x, T4n_res1[0].imag, label=R"$T_\mathrm{III}(s-0i)$")
ax4.plot(x, T4p_res1[0].imag, label=R"$T_\mathrm{IV}(s+0i)$")
ax4.set_title(R"$T_\mathrm{III}$")

for ax in axes:
    ax.legend()
    ax.set_ylim(-1, +1)

fig.tight_layout()
plt.show()

## Visualization of the Riemann sheets 2 dimensional  

In [ ]:
%config InlineBackend.figure_formats = ["png"]

fig, axes = plt.subplots(figsize=(12, 8), ncols=2, nrows=2, sharey=True)
ax1, ax2, ax3, ax4 = axes.flatten()

ax1.set_xlabel(R"$\mathrm{Re}\,s$")
ax1.set_ylabel(R"$\mathrm{Im}\,s$")

ax1.set_title("I and II")
ax2.set_title("I and III")
ax3.set_title("I and II")
ax4.set_title("I and III")

T_max = 2

style = dict(vmin=-T_max, vmax=+T_max, cmap=plt.cm.coolwarm)
mesh = ax1.pcolormesh(X, Y, T1p_res1.imag, **style)
ax1.pcolormesh(X, -Y, T2n_res1.imag, **style)
ax2.pcolormesh(X, +Y, T1p_res1.imag, **style)
ax2.pcolormesh(X, -Y, T3n_res1.imag, **style)
ax3.pcolormesh(X, +Y, T1p_res2.imag, **style)
ax3.pcolormesh(X, -Y, T2n_res2.imag, **style)
ax4.pcolormesh(X, +Y, T1p_res2.imag, **style)
ax4.pcolormesh(X, -Y, T3n_res2.imag, **style)

s_thr1 = values_1[ma1] + values_1[mb1]
s_thr2 = values_1[ma2] + values_1[mb2]
linestyle = dict(ls="dotted", lw=1)
for ax in axes.flatten():
    ax.axhline(0, c="black", **linestyle)
    ax.axvline(s_thr1, c="C0", **linestyle, label=R"$\sqrt{s_\text{thr1}}$")
    ax.axvline(s_thr2, c="C1", **linestyle, label=R"$\sqrt{s_\text{thr2}}$")
linestyle = dict(ls="dotted", lw=1)
for ax in axes[0]:
    ax.axvline(values_1[m0], c="r", ls="dotted", label=R"$m_\text{res}$")
for ax in axes[1]:
    ax.axvline(values_2[m0], c="r", ls="dotted", label=R"$m_\text{res}$")
ax2.legend()

fig.text(0.5, 0.93, R"$s_{thr1}<s_{thr2}<m_{res}$", ha="center", fontsize=18)
fig.text(0.5, 0.46, R"$s_{thr1}<m_{res}<s_{thr2}$", ha="center", fontsize=18)
plt.subplots_adjust(wspace=1)
cbar = fig.add_axes([0.92, 0.15, 0.02, 0.7])
plt.colorbar(mesh, cax=cbar)

fig.tight_layout(rect=[0, 0.03, 0.9, 0.95])

plt.show()

It can be shown that if the resonance mass is above both thresholds the third sheet connects smoothly to the first sheet. If the resonance mass is above the first and below the second threshold the second sheet transitions smoothly into the first sheet.

## Visualization of the Riemann sheets 3 dimensional

In [ ]:
def sty(sheet_name: str) -> dict:
    sheet_color = sheet_colors[sheet_name]
    n_lines = 16
    return dict(
        cmin=-vmax,
        cmax=+vmax,
        colorscale=[[0, "rgb(0, 0, 0)"], [1, sheet_color]],
        contours=dict(
            x=dict(
                show=True,
                start=x.min(),
                end=x.max(),
                size=(x.max() - x.min()) / n_lines,
                color="black",
            ),
            y=dict(
                show=True,
                start=-y.max(),
                end=+y.max(),
                size=(y.max() - y.min()) / (n_lines // 2),
                color="black",
            ),
        ),
        name=sheet_name,
        opacity=0.4,
        showscale=False,
    )


vmax = 2.0
project = np.imag
sheet_colors = {
    "T1 (physical)": "blue",
    "T2 (unphysical)": "red",
    "T3 (unphysical)": "green",
    "T4 (unphysical)": "yellow",
}

In [ ]:
Sn_II_res1 = go.Surface(x=X, y=-Y, z=T2n_res1.imag, **sty("T2 (unphysical)"))
Sn_III_res1 = go.Surface(x=X, y=-Y, z=T3n_res1.imag, **sty("T3 (unphysical)"))
Sp_res1 = go.Surface(x=X, y=+Y, z=T1p_res1.imag, **sty("T1 (physical)"))

Sn_II_res2 = go.Surface(x=X, y=-Y, z=T2n_res2.imag, **sty("T2 (unphysical)"))
Sn_III_res2 = go.Surface(x=X, y=-Y, z=T3n_res2.imag, **sty("T3 (unphysical)"))
Sp_res2 = go.Surface(x=X, y=+Y, z=T1p_res2.imag, **sty("T1 (physical)"))

thr1_filter = x >= s_thr1
thr2_filter = x >= s_thr2

line_kwargs = dict(
    line=dict(color="yellow", width=10),
    mode="lines",
    name="Lineshape",
)
thr1_lineshape1 = go.Scatter3d(
    x=x[thr1_filter],
    y=np.zeros(thr1_filter.shape),
    z=project(T1p_res1[0])[thr1_filter],
    **line_kwargs,
)
thr2_lineshape1 = go.Scatter3d(
    x=x[thr2_filter],
    y=np.zeros(thr2_filter.shape),
    z=project(T1p_res2[0])[thr2_filter],
    **line_kwargs,
)
thr1_lineshape2 = go.Scatter3d(
    x=x[thr1_filter],
    y=np.zeros(thr1_filter.shape),
    z=project(T1p_res1[0])[thr1_filter],
    **line_kwargs,
)
thr2_lineshape2 = go.Scatter3d(
    x=x[thr2_filter],
    y=np.zeros(thr2_filter.shape),
    z=project(T1p_res2[0])[thr2_filter],
    **line_kwargs,
)

point_kwargs = dict(
    marker=dict(color="black", size=4),
    mode="markers",
    name="Branch point",
)
thr1_point = go.Scatter3d(x=[s_thr1], y=[0], z=[0], **point_kwargs)
thr2_point = go.Scatter3d(x=[s_thr2], y=[0], z=[0], **point_kwargs)

fig = make_subplots(
    rows=2,
    cols=2,
    horizontal_spacing=0.01,
    vertical_spacing=0.05,
    specs=[
        [{"type": "surface"}, {"type": "surface"}],
        [{"type": "surface"}, {"type": "surface"}],
    ],
    subplot_titles=[
        R"Sheet I and II thr₁ < thr₂ < mᵣ",
        R"Sheet I and II thr₁ < thr₂ < mᵣ",
        R"Sheet I and II thr₁ < mᵣ < thr₂",
        R"Sheet I and III thr₁ < mᵣ < thr₂",
    ],
)

selector = dict(row=1, col=1)
fig.add_trace(Sp_res1, **selector)
fig.add_trace(Sn_III_res1, **selector)
fig.add_trace(thr1_lineshape1, **selector)
fig.add_trace(thr1_point, **selector)

selector = dict(row=1, col=2)
fig.add_trace(Sp_res1, **selector)
fig.add_trace(Sn_II_res1, **selector)
fig.add_trace(thr1_lineshape2, **selector)
fig.add_trace(thr1_point, **selector)

selector = dict(row=2, col=1)
fig.add_trace(Sp_res2, **selector)
fig.add_trace(Sn_II_res2, **selector)
fig.add_trace(thr2_lineshape1, **selector)
fig.add_trace(thr2_point, **selector)

selector = dict(col=2, row=2)
fig.add_trace(Sp_res2, **selector)
fig.add_trace(Sn_III_res2, **selector)
fig.add_trace(thr2_lineshape2, **selector)
fig.add_trace(thr2_point, **selector)

fig.update_layout(
    margin=dict(l=0, r=0, t=20, b=0),
    height=600,
    showlegend=False,
)

fig.update_scenes(
    camera_center=dict(z=-0.1),
    camera_eye=dict(x=1.4, y=1.4, z=1.4),
    xaxis_title_text="Re s",
    yaxis_title_text="Im s",
    zaxis_title_text="Im T(s)",
    zaxis_range=[-vmax, +vmax],
)
fig.show()